In [15]:
# Import dependencies
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import balanced_accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
import pickle

In [16]:
# Read data
loan_train_df = pd.read_csv("Dataset\loan_sanction_train.csv")

In [17]:
# Clean dataset and convert categorical to numeric
loan_train_clean_df = loan_train_df.drop(columns=["Loan_ID"])
loan_train_clean_df = pd.get_dummies(loan_train_clean_df, columns=["Gender",
                                                                   "Married",
                                                                   "Dependents",
                                                                   "Education",
                                                                   "Self_Employed",
                                                                   "Property_Area"])
loan_train_clean_df = loan_train_clean_df.dropna()

In [18]:
loan_train_clean_df

,ApplicantIncome,CoapplicantIncome,LoanAmount,Loan_Amount_Term,Credit_History,Loan_Status,Gender_Female,Gender_Male,Married_No,Married_Yes,...,Dependents_1,Dependents_2,Dependents_3+,Education_Graduate,Education_Not Graduate,Self_Employed_No,Self_Employed_Yes,Property_Area_Rural,Property_Area_Semiurban,Property_Area_Urban
1,4583,1508.0,128.0,360.0,1.0,N,False,True,False,True,...,True,False,False,True,False,True,False,True,False,False
2,3000,0.0,66.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,True,False,False,True,False,False,True
3,2583,2358.0,120.0,360.0,1.0,Y,False,True,False,True,...,False,False,False,False,True,True,False,False,False,True
4,6000,0.0,141.0,360.0,1.0,Y,False,True,True,False,...,False,False,False,True,False,True,False,False,False,True
5,5417,4196.0,267.0,360.0,1.0,Y,False,True,False,True,...,False,True,False,True,False,False,True,False,False,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
609,2900,0.0,71.0,360.0,1.0,Y,True,False,True,False,...,False,False,False,True,False,True,False,True,False,False
610,4106,0.0,40.0,180.0,1.0,Y,False,True,False,True,...,False,False,True,True,False,True,False,True,False,False
611,8072,240.0,253.0,360.0,1.0,Y,False,True,False,True,...,True,False,False,True,False,True,False,False,False,True
612,7583,0.0,187.0,360.0,1.0,Y,False,True,False,True,...,False,True,False,True,False,True,False,False,False,True


In [19]:
# (X,Y) data labels 
X = loan_train_clean_df.drop(columns=["Loan_Status"]).values
y = loan_train_clean_df["Loan_Status"].values

In [20]:
# Encode the target variable (Loan_Status) to numeric values
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [21]:
# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1, stratify=y)

X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [22]:
# Creating the deep learning model
number_input_features = len(X_train_scaled[0])
hidden_nodes_layer1 =  64
hidden_nodes_layer2 = 32
hidden_nodes_layer3 = 1


nn = tf.keras.models.Sequential()

# First hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer1, input_dim=number_input_features, activation="relu"))

# Second hidden layer
nn.add(tf.keras.layers.Dense(units=hidden_nodes_layer2, activation="relu"))

# Output layer
nn.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

# Check the structure of the model
nn.summary()

c:\Users\A1990\anaconda3\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [23]:
# Compile the model
nn.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
nn.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_split=0.2)

Epoch 1/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 1s 20ms/step - accuracy: 0.6000 - loss: 0.6946 - val_accuracy: 0.6125 - val_loss: 0.6968
Epoch 2/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6971 - loss: 0.6300 - val_accuracy: 0.6125 - val_loss: 0.6641
Epoch 3/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.6922 - loss: 0.6131 - val_accuracy: 0.6125 - val_loss: 0.6368
Epoch 4/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7665 - loss: 0.5381 - val_accuracy: 0.6375 - val_loss: 0.6145
Epoch 5/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.7452 - loss: 0.5423 - val_accuracy: 0.6875 - val_loss: 0.5968
Epoch 6/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8050 - loss: 0.4974 - val_accuracy: 0.7250 - val_loss: 0.5818
Epoch 7/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8252 - loss: 0.4666 - val_accuracy: 0.7375 - val_loss: 0.5680
Epoch 8/50
10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.8252 - loss: 0.4376 - val_accuracy: 0.7375 - val_loss

In [24]:
# Evaluate the model on test data
loss, accuracy = nn.evaluate(X_test_scaled, y_test, verbose=2)
print(f"Loss: {loss}, Accuracy: {accuracy}")

5/5 - 0s - 5ms/step - accuracy: 0.7820 - loss: 0.5703
Loss: 0.5702748894691467, Accuracy: 0.7819548845291138


In [25]:
# Save the model to a file
filename = 'deep_learning_model.h5'
nn.save(filename)

# Load the model from the file and check its accuracy
loaded_model = tf.keras.models.load_model(filename)
loss, accuracy = loaded_model.evaluate(X_test_scaled, y_test)
print("Loaded Model Test Accuracy:", accuracy)

5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.8188 - loss: 0.4720
Loaded Model Test Accuracy: 0.7819548845291138
